In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.linear_model import (HuberRegressor, TheilSenRegressor, 
                                  LinearRegression as SkLinearRegression)
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import normalize
from abess.linear import LinearRegression
from bess import PdasLm
import pysindy as ps
from utils import BIC_AIC

import numpy as np; np.random.seed(0)

# from pde_diff import *
from pde_diff_new import * # moving to the newer version of PDE-FIND

from parametric_pde_find import *
# from robust_pde_diff import DLrSR, RobustPCA
# https://github.com/nerdull/denoise
# from denoise import Denoiser, kalman_denoise
from tvregdiff import TVRegDiff, tvregdiff, numdiff, pysindydiff
import pynumdiff
import sys; sys.path.insert(0, "../derivative/"); import derivative

from RobustPCA.rpca import RobustPCA
from RobustPCA.spcp import StablePCP
# from hyperspy.signals import Signal1D
from r_pca import R_pca

from scipy.integrate import odeint
from scipy.linalg import block_diag
from scipy.special import huber as hb
from scipy.signal import wiener, savgol_filter # (+0, +1)
from numpy.fft import fft, ifft, fftfreq
from best_subset import *
import statsmodels.api as sm
# from pysr import PySRRegressor

import random; SEEED = 0; random.seed(SEEED)
from random import randint, sample
from tqdm import trange, tqdm
from time import time
from functools import cmp_to_key

Running Python 3.7.10
You can use npar for np.array


In [2]:
def parametric_burgers_rhs(u, t, params):
    k,a,b,c = params
    deriv = a*(1+c*np.sin(t))*u*ifft(1j*k*fft(u)) + b*ifft(-k**2*fft(u))
    return deriv.real

In [3]:
# Set size of grid
n = 256
m = 256

# Set up grid
x = np.linspace(-8,8,n+1)[:-1];   dx = x[1]-x[0]
t = np.linspace(0,10,m);          dt = t[1]-t[0]
k = 2*np.pi*fftfreq(n, d = dx)

# Initial condition
u0 = np.exp(-(x+1)**2)

# Solve with time dependent uu_x term
params = (k, -1, 0.1, 0.25)
u = odeint(parametric_burgers_rhs, u0, t, args=(params,)).T

u_xx_true = 0.1*np.ones(m)
uu_x_true = -1*(1+0.25*np.sin(t))

In [4]:
# # Plot
# fig=figure(figsize=(16,4))
# X, T = np.meshgrid(x, t)

# subplot(1,2,1)
# pcolor(X, T, u.T, cmap=cm.coolwarm)
# xlabel('x', fontsize = fontsize)
# ylabel('t', fontsize = fontsize)
# xticks(fontsize = fontsize)
# yticks(fontsize = fontsize)
# xlim([x[0],x[-1]])

# subplot(1,2,2)
# plot(t, uu_x_true, label=r'$uu_{x}$')
# plot(t, u_xx_true, label=r'$u_{xx}$')

# xticks(fontsize = fontsize)
# yticks(fontsize = fontsize)
# xlabel('t', fontsize = fontsize)
# legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize = fontsize+2)
# fig.tight_layout()

#### Computing dervatives + std Noise

In [5]:
# just for the reference
u_clean = u.copy()
Ut_clean, Theta_clean, rhs_des_clean = build_linear_system(u_clean, dt, dx, D=4, P=3, time_diff = 'FD', space_diff = 'FD')
# np.allclose(Theta_clean[:, 1:2], u_clean.T.flatten().reshape(-1, 1)) == True
Theta_grouped_clean = [(Theta_clean[j*n:(j+1)*n,:]).real for j in range(m)]
Ut_grouped_clean = [(Ut_clean[j*n:(j+1)*n]).real for j in range(m)]
# finitediff_x = Theta_grouped[1][:, 3:4].flatten()

noise_lv = 90; noise = 0.01*np.abs(noise_lv)*u.std()*np.random.randn(n,m)
u = u + noise

if np.abs(noise_lv) > 0:
    # Build linear systems
    # D=3 and p=2 for the noise-aware physics-informed paper
    wx = 10; wt = 10
    Ut, Theta, rhs_des = build_linear_system(u, dt, dx, D=4, P=3, time_diff = 'poly',
                                           deg_x = 6, deg_t = 4, 
                                           width_x = wx, width_t = wt)
    n = n - 2*wx
    m = m - 2*wt
else:
    wx = 0; wt = 0
    Ut, Theta, rhs_des = build_linear_system(u, dt, dx, D=4, P=3, time_diff = 'FD', space_diff = 'FD')

# removing the constant term...
Theta = Theta[:, 1:]; rhs_des = rhs_des[1:]
# Group by timestep
Theta_grouped = [(Theta[j*n:(j+1)*n,:]).real for j in range(m)]
Ut_grouped = [(Ut[j*n:(j+1)*n]).real for j in range(m)]

In [6]:
# Define weak form PDE library 
library_functions = [lambda x: x, lambda x: x * x]
library_function_names = [lambda x: x, lambda x: x + x]

# Need to define the 2D spatiotemporal grid before calling the library
X, T = np.meshgrid(x, t)
XT = np.asarray([X, T]).T
pde_lib = ps.WeakPDELibrary(library_functions=library_functions, 
                            function_names=library_function_names, 
                            derivative_order=2,
                            spatiotemporal_grid=XT,
                            is_uniform=True, K=1000,
                            )

In [7]:
mse(ps.SmoothedFiniteDifference(axis=0)._differentiate(u, t=dt).flatten(), Ut_clean)

0.778807566767364

In [8]:
optimizer = ps.SR3(threshold=0.1, thresholder='l0', 
                   tol=1e-8, normalize_columns=True, max_iter=1000)
model = ps.SINDy(feature_library=pde_lib, optimizer=optimizer, differentiation_method=ps.SmoothedFiniteDifference, feature_names=['u'])
model.fit(np.expand_dims(u,-1), quiet=True)
model.print()

(u)' = -0.178 uu + -0.187 u_1 + -0.603 uu_1 + -0.009 uuu_11


In [9]:
model.get_feature_names()

['u', 'uu', 'u_1', 'u_11', 'uu_1', 'uuu_1', 'uu_11', 'uuu_11']